In [1]:
import torch
from TTS.api import TTS
from utils.preprocess import *
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

2024-10-11 14:40:42.478693: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
classifier = pipeline("sentiment-analysis", model="michellejieli/emotion_text_classifier")
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


 > tts_models/multilingual/multi-dataset/xtts_v2 has been updated, clearing model cache...
 > Downloading model to /Users/wangqiqian/Library/Application Support/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|██████████████████████████████████▉| 1.87G/1.87G [07:41<00:00, 4.09MiB/s]
100%|███████████████████████████████████| 1.87G/1.87G [07:42<00:00, 4.04MiB/s]
100%|███████████████████████████████████| 4.37k/4.37k [00:00<00:00, 10.1kiB/s]
 69%|██████████████████████████            | 248k/361k [00:00<00:00, 578kiB/s]
100%|██████████████████████████████████████| 361k/361k [00:00<00:00, 379kiB/s]
100%|█████████████████████████████████████| 32.0/32.0 [00:00<00:00, 19.0kiB/s]


 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


GPT2InferenceModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [3]:
with open("./Labels/70.txt", "r", encoding="utf-8") as file:
    text = file.read()
    
quoted_sentences = extract_sentences_and_narration(text)
classified_lst = [classifier(sentence['text']) for sentence in quoted_sentences]
# quoted_sentences = pd.DataFrame(quoted_sentences)
# classified_lst = [classifier(sentence) for sentence in pd.DataFrame(quoted_sentences)]

combined = [
    {'text': quoted_sentences[i]['text'],
     'narration': quoted_sentences[i]['narration'],
     'label': classified_lst[i][0]['label'], 
     'score': classified_lst[i][0]['score']}
    for i in range(len(quoted_sentences))
]
df_combined = pd.DataFrame(combined)
df_combined

,text,narration,label,score
0,"FAIR, BROWN, AND TREMBLING King Hugh Curucha ...",1,neutral,0.977213
1,"Fair and Brown had new dresses, and went to ch...",1,neutral,0.680441
2,Trembling was kept at home to do the cooking a...,1,neutral,0.977093
3,They would not let her go out of the house at ...,1,fear,0.954711
4,They carried on in this way for seven years,1,neutral,0.753016
...,...,...,...,...
196,If the little girl that is born to us now live...,0,neutral,0.584807
197,The cowboy and the prince's daughter lived on ...,1,neutral,0.829893
198,The mother said to her husband,1,neutral,0.956316
199,You could not have saved me from the whale but...,0,disgust,0.417073


In [5]:
# neutral if it's narration
for index, row in df_combined.iterrows():
    if row['narration']:
        tts.tts_to_file(
            text = row["text"],
            speaker_wav = "Emotions/record_out.wav",
            language = "en",
            file_path = f"Output/sentence_{index+1}.wav",
            emotion = 'neutral'
        )
    else:
        tts.tts_to_file(
            text = row["text"],
            speaker_wav = "Emotions/record_out.wav",
            language = "en",
            file_path = f"Output/sentence_{index+1}.wav",
            emotion = row["label"]
        )

In [6]:
from pydub import AudioSegment

wav_files = [f"Output/sentence_{i}.wav" for i in range(1, 5)]

combined_audio = AudioSegment.from_wav(wav_files[0])

# 逐步將剩下的音頻文件合併到第一個音頻文件中
for wav_file in wav_files[1:]:
    next_audio = AudioSegment.from_wav(wav_file)
    combined_audio += next_audio

# 將合併後的音頻文件保存
combined_audio.export("Output/combined_audio.wav", format="wav")

<_io.BufferedRandom name='Output/combined_audio.wav'>